# [Module 2] Monitor and Analyze Training Jobs Using Metrics

Amazon SageMaker 학습 작업은 모델이 학습 데이터셋에서 샘플들을 제시하여 예측하도록 가르치는 반복적인 프로세스입니다.
일반적으로 학습 알고리즘은 학습 오차(training error) 및 예측 정확도(prediction accuracy)와 같은 여러 지표들(metrics)을 계산합니다. 이러한 지표들은 모델이 잘 학습되고 있는지 확인하고 신규 데이터를 잘 예측할 수 있도록 일반화합니다.

학습 알고리즘은 이러한 지표들의 값을 Amazon SageMaker가 모니터링하고 실시간으로 Amazon CloudWatch로 전송하는 로그에 기록합니다.

Amazon SageMaker가 사용자 정의 알고리즘의 로그를 파싱하고 알고리즘이 생성하는 지표를 CloudWatch로 보내려면, 학습 작업을 설정할 때 Amazon SageMaker가 CloudWatch로 전송할 지표를 지정해야 합니다.<br>
이 때, 여러분은 전송하려는 지표들의 이름과 정규표현식들(regular expressions)을 지정해야 합니다. 정규표현식들은 Amazon SageMaker가 해당 지표를 찾기 위해 필요합니다.

본 실습에서는 Amazon SageMaker를 활용한 지표 모니터링에 대해 실시합니다. 실습 전반에서는 Amazon CloudWatch를 사용하여 모니터링을 실시합니다.
실습 후반에서는 Keras에서 출력하는 학습 로그를 활용하여 TensorBoard에서 학습 상황을 모니터링해 봅니다.

## Defining Training Metrics (Amazon SageMaker Python SDK)

Estimator 객체를 초기화할 때 지표 이름 및 정규식 목록을 metric_definitions 인수로 지정하여 CloudWatch로 보내려는 지표를 정의하세요. 예를 들어 CloudWatch에서 train:error 및 validation:error 지표를 모두 모니터링하려는 경우 Estimator 초기화 방법은 다음 코드와 같습니다.

```python 
estimator = Estimator(image_name=ImageName,
            role='SageMakerRole', train_instance_count=1,
            train_instance_type='ml.c4.xlarge',            
            k=10,
            metric_definitions=[
                   {'Name': 'train:error', 'Regex': 'Train_error=(.*?);'},
                   {'Name': 'validation:error', 'Regex': 'Valid_error=(.*?);'}
            ])
```

## Monitoring the CIFAR10 training
아래 작업들을 직접 수행해 보세요.
- SageMaker 콘솔에서 이전에 학습했던 학습 작업(cifar10_keras_sm)을 찾아 보세요.
- 작업 세부 사항(job details)을 열고 CloudWatch 로그를 확인합니다.
- 로그에 맞는 지표 정규식을 설정하세요. 정규식 도구들(regex tools)을 사용하여 정규식을 확인하고 ()를 사용하여 각 matric을 확인합니다.

In [1]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [3]:
prefix = 'data/DEMO-cifar10'
dataset_location = os.path.join('s3://', sagemaker_session.default_bucket(), prefix)

`0_Running_TensorFlow_In_SageMaker.ipynb`의 estimator 인스턴스 생성 코드를 그대로 복사 후에
`metric_definitions=metric_definitions` 인자를 추가해 주세요. 

In [12]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10-cloudwatch',
                       entry_point='cifar10_keras_sm.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions) # 추가

In [13]:
%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
2020-02-28 01:42:01 Starting - Starting the training job...
2020-02-28 01:42:02 Starting - Launching requested ML instances...
2020-02-28 01:42:59 Starting - Preparing the instances for training............
2020-02-28 01:44:36 Downloading - Downloading input data...
2020-02-28 01:45:27 Training - Training image download completed. Training in progress..2020-02-28 01:45:30,949 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-02-28 01:45:31,323 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperpara

### View the job training metrics
SageMaker는 위에서 구성한 정규식을 사용하여 작업 지표(job metrics)를 CloudWatch 지표로 전송했습니다.
이제 SageMaker 콘솔에서 직접 작업 지표을 보실 수 있습니다.

[SageMaker console](https://console.aws.amazon.com/sagemaker/home) 에 로그인하여 최신 학습 작업(latest training job)을 선택하고 모니터 섹션(monitor section)까지 아래로 스크롤하십시오.
CloudWatch 지표를 사용하여 기간(period)을 변경하고 통계치(statistics)들을 설정할 수 있습니다

다음 셀(cell)을 사용하여 지표를 찾아 보세요.

In [16]:
from IPython.core.display import Markdown

link = 'https://console.aws.amazon.com/cloudwatch/home?region='+sagemaker_session.boto_region_name+'#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20'+estimator.latest_training_job.job_name
display(Markdown('CloudWatch metrics: [link]('+link+')'))
display(Markdown('After you choose a metric, change the period to 1 Minute (Graphed Metrics -> Period)'))

CloudWatch metrics: [link](https://console.aws.amazon.com/cloudwatch/home?region=us-east-2#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20cifar10-cloudwatch-2020-02-28-01-42-00-902)

After you choose a metric, change the period to 1 Minute (Graphed Metrics -> Period)

In [17]:
display(Markdown('CloudWatch metrics: [link]('+link+')'))
display(Markdown('After you choose a metric, change the period to 1 Minute (Graphed Metrics -> Period)'))

CloudWatch metrics: [link](https://console.aws.amazon.com/cloudwatch/home?region=us-east-2#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20cifar10-cloudwatch-2020-02-28-01-42-00-902)

After you choose a metric, change the period to 1 Minute (Graphed Metrics -> Period)

## Monitor with TensorBoard

이번에는 TensorBoard로 학습 작업을 실시간으로 모니터링해 봅니다.<br>
TensorBoard는 머신 러닝 실험에 필요한 시각화 및 도구들을 제공합니다.
* Loss 및 accuracy과 같은 metric 추적 및 시각화
* 모델 그래프 (ops 및 layers) 시각화
* 시간 경에 따라 변화하는 가중치(weights), 바이어스(biases) 또는 기타 텐서의 히스토그램 확인
* 저차원 공간으로 임베딩(embedding)
* 이미지, 텍스트 및 오디오 데이터 표시
* 기타 

**`training_script/cifar10_keras_sm.py`에서 스크립트 사본을 생성 후, `training_script/cifar10_keras_tensorboard.py`로 저장하세요.**

스크립트 사본을 생성하였다면 단계별로 아래의 작업들을 직접 시도합니다.

----
### TODO 1.

먼저, 학습 로그를 TensorBoard로 전송하기 위해 `cifar10_keras_tensorboard.py` 스크립트를 수정합니다.<br>
Keras에서 TensorBoard를 사용하려면 코드 상단에 `from keras.callbacks import TensorBoard` 구문을 추가해 주세요.

Keras는 디폴트로 각 배치(batch)마다 TensorBoard 로그를 보냅니다. S3에 로그를 전송하면 학습 작업이 느려지기 때문에,
TensorBoard callback을 각 epoch의 끝에서만 로그를 전송할 수 있게 변경해 주는 것이 좋습니다.

----
### TODO 2.

스크립트에 TensorBoard callback을 추가해 주세요 (ModelCheckpoint callback 바로 다음 라인에 아래 줄을 추가해 주세요).
```python
callbacks.append(TensorBoard(log_dir=args.model_output_dir,update_freq='epoch'))
```

<font color='blue'>**본 노트북 실습에 어려움이 있다면 솔루션 파일 `training_script/cifar10_keras_tensorboard_solution.py`을 참조하시면 됩니다.**</font>

### Run a training job with TensorBoard support

In [23]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10-tensorboard',
                       entry_point='cifar10_keras_tensorboard.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions)

In [24]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)}, wait=False)
 # Use wait=False to run async jobs

### Install Tensorboard on your local machine

`pip install tensorboard`를 사용하여 [TensorBoard](https://github.com/tensorflow/tensorboard)를 로컬에 설치해 주세요.
S3 로그 디렉토리에 접근하기 위해 TensorBoard 기본 리젼을 설정해 주세요. 여러분은 `AWS_REGION`이라는 환경 변수를 설정하고 환경 변수의 값을 학습 작업이 실행되는 AWS 리전으로 설정하면 됩니다.
예를 들면 `AWS_REGION='us-east-1' tensorboard --logdir model_dir` 입니다.

**여러분은 다음 셀(cell)에서 TensorBoard command를 얻을 수 있습니다.**

You'll need and AccessKey + SecretKey with access to model_dir for this Event, get those from https://dashboard.eventengine.run/dashboard

In [25]:
!pip install tensorboard

You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
from IPython.core.display import Markdown

link = 'AWS_REGION=\''+sagemaker_session.boto_region_name+'\' tensorboard --logdir ' + estimator.model_dir + ' --host localhost --port 6006'
display(Markdown('본 셀의 output를 다음 셀에 붙여넣기 해서 TensorBoard를 시작해 주세요.'))
display(Markdown('!'+link))

본 셀의 output를 다음 셀에 붙여넣기 해서 TensorBoard를 시작해 주세요.

!AWS_REGION='us-east-2' tensorboard --logdir s3://sagemaker-us-east-2-057716757052/cifar10-tensorboard-2020-02-28-02-25-58-733/model --host localhost --port 6006

**잘 하셨습니다.**

이제 여러분의 학습 작업을 CloudWatch 지표와 TensorBoard로 확인할 수 있습니다.<br>
다음 노트북으로 계속 진행하기 전에 다른 TensorBoard 설정값들을 [TensorBoard callback configuration](https://keras.io/callbacks/#tensorboard)에서 살펴보세요.